In [1]:
from __future__ import division
import numpy as np
import scipy as sp
import cPickle
from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

<h3>Linear Models for Classification</h3>
For this notebook I will only consider the simpliest possible linear models, of the form

$$ f(x_i, W, b) = Wx_i^T + b $$

Where $x_i$ is a single data point $x_i = [x_i^1, x_i^2, ..., x_i^P]$, $W$ is a matrix of weights of shape $10 \times P $ and b is a vector of shape $10 \times 1$.

First we'll load in our image data, then run some simple preprocessing on it so that all pixel values are between $-1$ and $1$.

In [2]:
files = ['cifar-10-batches-py/data_batch_' + str(i) for i in range(1, 6)]
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

batches = map(unpickle, files)
X = np.concatenate([b['data'] for b in batches])
X = (X.astype(float) - 127) / 255
y = np.concatenate([b['labels'] for b in batches])

X_test = map(unpickle, ['cifar-10-batches-py/test_batch'])[0]['data']
X_test = (X_test.astype(float) - 127) / 255
y_test = map(unpickle, ['cifar-10-batches-py/test_batch'])[0]['labels']

In [3]:
X_test

array([[ 0.12156863,  0.1254902 ,  0.14901961, ..., -0.01176471,
         0.00784314, -0.06666667],
       [ 0.42352941,  0.40784314,  0.41176471, ...,  0.2       ,
         0.25098039,  0.28235294],
       [ 0.12156863,  0.12156863,  0.04705882, ..., -0.46666667,
        -0.48627451, -0.47058824],
       ..., 
       [-0.41960784, -0.42352941, -0.43921569, ..., -0.30196078,
        -0.29019608, -0.31372549],
       [-0.4       , -0.43921569, -0.40784314, ..., -0.18431373,
        -0.18039216, -0.18431373],
       [-0.21176471, -0.11372549, -0.10980392, ..., -0.12941176,
        -0.27058824, -0.39607843]])

<h3>Multiclass Support Vector Machine Loss</h3>
In order to evaluate a linear model, we'll need a loss function to choose our parameters.  The Multiclass SVM loss is set up so that it "wants" to have the correct class score higher than the other scores by a certain margin.

For the $i^{th}$ training example we are given a vector of data $x_i$ and label $y_i$.  The score function takes these and computes $f(x_i, y_i)$, the vector of class scores.  For each class in our class vector that is not the correct class $y_i$, we take the difference between the score for the correct class $y_i$ and for the current class $j$ plus a margin $\Delta$.


$$L_i = \sum_{j \neq y_i} max(0, f(x_i, W)_j - f(x_i, W)_{y_i} + \Delta) $$

For example, if $\Delta = 10$ and there are three classes, the above might be

\begin{align}
L_i &= max(0, 6 - 12 + 10) + max(0, 1 - 12 + 10)\\
L_i &= 4
\end{align}

Essentially what were are doing is checking if the different between the correct class and all other classes is at least $\Delta$; beyond this point we do not care - the loss is always cutoff at 0.  The threshold at 0 is often called the Hinge Loss.  Sometimes people refer to the squared hinge loss (L2-SVM) which is $max(0, -)^2$

<h3>Regularization</h3>
The one problem with the loss function presented above is that the vector $W$ which correctly classifies all values may not be unique.  In order to express our preference for matricies with smaller weights, we will include regularization to shrink the weight sizes.  The most common would be $L2$ regularization - an elementwise penalty across all weights

$$R(W) = \sum_l \sum_k w_{k, l}^2 $$

The complete loss function

$$L = \frac{1}{N}\sum_{i}L_i + \lambda R(W) $$


Now for a fully vectorized implementation in python (that is, no python for loops).  Where I have a dataset X of size $50000 \times 3072$ and a weight vector of size $10 \times 3072$ and a bias vector of size $10 \times 1$

In [4]:
W = 0.01 * np.random.randn(10, 3072)
W

array([[-0.01161161, -0.00971034,  0.00239516, ..., -0.00631566,
        -0.02677543, -0.00561924],
       [-0.01272528, -0.01121538, -0.0129076 , ..., -0.0046118 ,
         0.02102181,  0.00360619],
       [-0.00800197, -0.0096109 ,  0.01541131, ...,  0.01569905,
         0.00799403, -0.00427914],
       ..., 
       [ 0.00979782, -0.00653428,  0.00300039, ..., -0.01664638,
         0.00333352, -0.01273976],
       [ 0.01605521,  0.00730659, -0.00376265, ..., -0.01462053,
         0.0076531 , -0.01306435],
       [ 0.00143793,  0.00062906,  0.00897753, ...,  0.00593007,
        -0.00381795,  0.00241182]])

In [5]:
b = 0.01 * np.random.randn(10, 1)
b

array([[ 0.00742199],
       [-0.01429153],
       [ 0.01937989],
       [-0.00217316],
       [ 0.00604422],
       [ 0.00149876],
       [ 0.01263421],
       [-0.01426743],
       [ 0.00351166],
       [ 0.00139518]])

In [6]:
L = np.dot(W, X.T) + b

In [7]:
L.shape

(10, 50000)

In the above we have sucessfully calculated the SVM scores with the given weight and bias vector using only linear algebra operations - now to implement the SVM loss.  Where y is the correct class, it is also the index of the score vector we wish to ignore.  We need to convert this array into a binary mask and then set all of the selected indexes of L to 0

In [59]:
L.T[[i for i in xrange(len(L[0]))], y]

array([ 0.04111017,  0.13602194,  0.02151653, ..., -0.12320148,
        0.03972904, -0.03654969])

In [60]:
L.T[0]

array([-0.06576129, -0.06460448,  0.14917444, -0.07659875,  0.12367278,
        0.04947665,  0.04111017,  0.07135378,  0.01407084, -0.03879697])

In [61]:
y[0]

6

Sweet it worked!  Now we just calcluate the SVM loss over all the examples with some margin $\Delta$ and then set the above indexed losses to 0

In [62]:
Correct = L.T[[i for i in xrange(len(L[0]))], y]

In [72]:
R = L.T - Correct[:, np.newaxis] + 10
R[[i for i in xrange(len(R))], y] = 0
np.sum(R, axis=1)

array([ 89.79199547,  89.47713091,  89.58325037, ...,  91.69389299,
        90.04576641,  89.99991378])

Now we can calculate the loss for a particular example as well as for the entire function - of course the randomly selected values for the weights don't work well!

<h3>Softmax Classifier</h3>
One of the limitations of the SVM classifier is that the class scores returned are difficult to interpret - for instance if you are classifying Dog, Cat, Bird and your class scores are 10, -2, 3.  While you understand that Dog is the selected class - it is difficult to compare the scores beyond this.

The softmax classifier alleviates this isssue by returning confidence scores - pseudo probabilities $\in [0, 1]$, replacing the hinge loss with the cross entropy

$$L_i = log(\frac{e^{f_{y_i}}}{\sum_i e^{f_{y_i}}}) $$ or equivalently

$$L_i = -f_{y_i} + \sum_i log(e^{f_{y_i}}) $$

The cross entropy between a true distribution $p$ and an estimated distribution $q$ can be measured by the cross entropy,

$$H(p, q) = - \sum_x p(x) logq(x) $$

The softmax classifier is therefore minimizing the distance between the normalized class scores from the softmax function and the correct label with probability mass entirely on the correct score $[0,0,0,...,1,0]$


<h3>Numerical Stability</h3>
The intermediate terms $e^f_{y_i}$ can potentially be very large, resulting in numerical problems when dividing.  To correct for this (without influenceing the results) you can multiply the top and bottom of the softmax function by some constant $C$, usually set to $C = -max(f_{y_i})$

\begin{align}
\frac{e^f_{y_i}}{\sum_j e^f_{y_i}} &= \frac{C e^f_{y_i}}{C \sum_j e^f_{y_i}} \\
&=  \frac{e^{f_{y_i} + log(C) }}{\sum_j e^{f_{y_i} + log(C)}}
\end{align}

The final result is unchanged but numerical stability is preserved

In [73]:
f = np.array([123, 456, 789])
np.exp(f) / np.sum(np.exp(f))

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: overflow encountered in exp
  from IPython.kernel.zmq import kernelapp as app
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: RuntimeWarning: invalid value encountered in true_divide
  from IPython.kernel.zmq import kernelapp as app


array([  0.,   0.,  nan])

In [74]:
f -= np.max(f)
np.exp(f) / np.sum(np.exp(f))

array([  5.75274406e-290,   2.39848787e-145,   1.00000000e+000])